<h1> Assignment Description </h1>
<p>In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.</p>

<p>For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format like the New York dataset.</p>

<p>Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.</p>

<h2> Importing Required Libraries<h2>

In [2]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
import matplotlib.pyplot
import seaborn as sns
import json 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /Users/zawtoemyint/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be SUPERSEDED by a higher-priority channel:

  conda               pkgs/main::conda-4.9.2-py37hecd8cb5_0 --> conda-forge::conda-4.9.2-py37hf985489_0



geopy-2.1.0          | 64 KB     | ###############################

In [10]:
!pip install wikipedia
import wikipedia as wp

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11685 sha256=fea535babea1c27e20123982bdc98703f8e31d17c2283bab815e619b2f024423
  Stored in directory: /Users/zawtoemyint/Library/Caches/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
You should consider upgrading via the '/Users/zawtoemyint/opt/anaconda3/bin/python3 -m pip install --upgrade pip' command.


<h2> Import Raw Data from Wikipedia</h2>

In [28]:
path = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df_wiki = pd.read_html(path)[0]
df_wiki.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h2> Remove Not-Assigned Data from Dataframe</h2>

In [30]:
neighborhood=df_wiki
neighborhood.rename(columns={0:'Postcode', 1: 'Borough', 2: 'Neighborhood'}, inplace=True)
neighborhood=neighborhood[neighborhood.Borough != 'Not assigned']
neighborhood=pd.DataFrame(neighborhood)
neighborhood.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
print("Shape: ", neighborhood.shape)

Shape:  (103, 3)


<h2> Merge with Geocoordinates</h2>

In [37]:
df_geo=pd.read_csv("/Users/zawtoemyint/OneDrive/Learning and Reading/IT_Data Science_IBM/9. Applied Data Science Capstone/Geospatial_Coordinates.csv")
frames=[neighborhood,df_geo]
frames=pd.concat(frames, axis=1, sort=False)
merge_columns=pd.merge(neighborhood, df_geo)
merge_columns.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
